# Prepare the data set
The goal is to prepare a folder structure like this (where which img is a 2D image)




```
data_yapic/
        └───input/
        │       img1.tiff
        │       img2.tiff
        │       img3.tiff
        │       ...
        │   
        └───target/
        │       img1_mask.tiff
        │       img2_mask.tiff
        │       img3_mask.tiff
        │       ...
```

In [2]:
import os
import numpy as np

from pathlib import Path
from tifffile import imread, imwrite

# Create the input dataset

In [4]:
im = imread('../data/tricho_8_contrast.tif')
values = [x for x in range(np.shape(im)[0])]

#for i in np.random.choice(values,10):
    #imwrite(f'../data/annotation/slice{i}.tif',im[i,:,:])

### From partial annotated tiff stack
For example using the `visualization_napari_workflow.ipynb`

# load raw image
path_im = Path('../data/louise/chip-2.tif')
im = imread(path_im)

# load label image
path_label = Path('../data/louise/chip-2-labels.tif')
labels = imread(path_label)

In [14]:
# load raw image
path_im = Path('../data/data_yapic/input/slice62.tif')
im = imread(path_im)

# load label image
path_label = Path('../data/data_yapic/target/slice_62_mask.tif')
labels = imread(path_label)

In [5]:
# create empty folder if it doesn't exist to save the data
dataset_path = Path('../data/data_louise_yapic')
if not os.path.isdir(dataset_path):
    os.mkdir(dataset_path)
    os.mkdir(dataset_path/"input")
    os.mkdir(dataset_path/"target")

In [11]:
dataset_path = Path('../data/data_yapic')

In [8]:
# populate the folders

for i, t in enumerate(labels):
    if 1 in t: # if this slide has labels
        # t[t!=1]=2
        imwrite(dataset_path/"input"/(str(i)+".tif"), im[i].astype(np.uint8), dtype=np.uint8)
        imwrite(dataset_path/"target"/(str(i)+".tif"), t.astype(np.uint8), dtype=np.uint8)

In [31]:
# check results
import napari
viewer = napari.Viewer()

# Train YAPIC Model

In [10]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

ModuleNotFoundError: No module named 'tensorflow'

In [1]:
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

2022-07-04 23:42:30.460539: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-07-04 23:42:30.463451: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-04 23:42:30.463461: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


[]


2022-07-04 23:42:31.506126: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-04 23:42:31.506606: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-04 23:42:31.506664: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-07-04 23:42:31.506714: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-07-04 23:42:31.508656: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

In [1]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2022-07-04 23:41:20.181031: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-07-04 23:41:20.183881: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-04 23:41:20.183890: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17863907655473116994
xla_global_id: -1
]


2022-07-04 23:41:21.227824: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-04 23:41:21.281605: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-04 23:41:21.282024: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-04 23:41:21.282085: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared o

## Augment the dataset

In [9]:
def random_fliprot(img, mask): 
    assert img.ndim >= mask.ndim
    axes = tuple(range(mask.ndim))
    perm = tuple(np.random.permutation(axes))
    img = img.transpose(perm + tuple(range(mask.ndim, img.ndim))) 
    mask = mask.transpose(perm) 
    for ax in axes: 
        if np.random.rand() > 0.5:
            img = np.flip(img, axis=ax)
            mask = np.flip(mask, axis=ax)
    return img, mask 

def random_intensity_change(img):
    img = img*np.random.uniform(0.6,2) + np.random.uniform(-0.2,0.2)
    return img


def augmenter(x, y):
    """Augmentation of a single input/label image pair.
    x is an input image
    y is the corresponding ground-truth label image
    """
    x, y = random_fliprot(x, y)
    x = random_intensity_change(x)
    # add some gaussian noise
    sig = 0.02*np.random.uniform(0,1)
    x = x + sig*np.random.normal(0,1,x.shape)
    return x, y

In [26]:
images = []
path_im = Path('../data/data_yapic/input')
path_label = Path('../data/data_yapic/target')
labels = []

for i,j in zip(os.listdir(path_im),os.listdir(path_label)):
    images.append(imread(str(path_im)+'/'+i))
    labels.append(imread(str(path_label)+'/'+j))

print(np.shape(images))

(10, 576, 576)


In [35]:
from random import randint

for i in range(30):
    v = randint(0, len(images)-1)
    imwrite(str(path_im)+'/slice_a_'+str(i)+'.tif',augmenter(images[v],labels[v])[0])
    imwrite(str(path_label)+'/slice_a_'+str(i)+'.tif',augmenter(images[v],labels[v])[1])


In [36]:
from yapic.session import Session

img_path = str(path_im)+'/*.tif'
label_path = str(path_label)+'/*.tif'

model_size_xy = (1, 576, 576)

t = Session()
t.load_training_data(img_path, label_path)
t.make_model('unet_2d', model_size_xy)

t.define_validation_data(0.2) # 80% training data, 20% validation data

t.train(max_epochs=10,
        steps_per_epoch=48,
        log_filename='../models/log.csv')

t.model.save('../models/my_model.h5')


ModuleNotFoundError: No module named 'yapic'